In [ ]:
pip install openpyxl pandas numpy sklearn matplotlib seaborn

In [ ]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import scatter_matrix
from sklearn.neighbors import LocalOutlierFactor
from sklearn import preprocessing

In [ ]:
x_bp = pd.read_excel(io = "./hw_data_composite/X_bp.xlsx")
x_bp = x_bp.astype({'Unnamed: 0':'int'})
x_bp

In [ ]:
x_nup = pd.read_excel(io = "./hw_data_composite/X_nup.xlsx")
x_nup = x_nup.astype({'Unnamed: 0':'int','Угол нашивки, град':'int'})
x_nup

In [ ]:
dataset = pd.merge(x_bp, x_nup, how='inner', on='Unnamed: 0')
dataset = dataset.drop(labels = 'Unnamed: 0', axis = 1)
dataset

In [ ]:
dataset.median()

In [ ]:
dataset.mean()

In [ ]:
ds_without_angles = dataset.drop(axis = 1, labels = 'Угол нашивки, град')

In [ ]:
ds_without_angles.hist(figsize = (20, 20), bins = 20, color = 'green')

In [ ]:
for col in ds_without_angles.columns:
    sns.boxplot(x = dataset[col], color = 'green')
    plt.show()

In [ ]:
pair_diagram = scatter_matrix(ds_without_angles, figsize = (16, 16), alpha = 0.25, color = 'green', diagonal = None)
for ax in pair_diagram.ravel():
    ax.set_xlabel(ax.get_xlabel(), fontsize = 10, rotation = 45)
    ax.set_ylabel(ax.get_ylabel(), fontsize = 10, rotation = 45)
pair_diagram

In [ ]:
dataset.isnull().sum()

In [ ]:
lof = LocalOutlierFactor(n_neighbors = 300)
y_pred = lof.fit_predict(ds_without_angles)
outlier_indexes = []
for i in range(len(y_pred)):
    if y_pred[i] == -1:
        outlier_indexes.append(i)
print('Кол-во выбросов: ' + str(len(outlier_indexes)))

In [ ]:
dataset.drop(outlier_indexes, inplace = True, errors='ignore')
ds_without_angles.drop(outlier_indexes, inplace = True, errors='ignore')
dataset

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()
dataset_normalized = pd.DataFrame(min_max_scaler.fit_transform(dataset.values))
dataset_normalized.columns = dataset.columns
dataset_normalized

In [ ]:
standart_scaler = preprocessing.StandardScaler()
dataset_standartized = pd.DataFrame(standart_scaler.fit_transform(dataset.values))
dataset_standartized.columns = dataset.columns
dataset_standartized

In [ ]:
dataset_0 = []
dataset_90 = []
for i in range(len(dataset)):
    if dataset['Угол нашивки, град'][i] == 0:
        dataset_90.append(i)
    else:
        dataset_0.append(i)
dataset_90 = dataset.drop(dataset_90, axis = 0)
dataset_0 = dataset.drop(dataset_0, axis = 0)

In [ ]:
pair_diagram = scatter_matrix(dataset_0, figsize = (16, 16), alpha = 0.25, color = 'green', diagonal = None)
for ax in pair_diagram.ravel():
    ax.set_xlabel(ax.get_xlabel(), fontsize = 10, rotation = 45)
    ax.set_ylabel(ax.get_ylabel(), fontsize = 10, rotation = 45)
pair_diagram

In [ ]:
pair_diagram = scatter_matrix(dataset_90, figsize = (16, 16), alpha = 0.25, color = 'green', diagonal = None)
for ax in pair_diagram.ravel():
    ax.set_xlabel(ax.get_xlabel(), fontsize = 10, rotation = 45)
    ax.set_ylabel(ax.get_ylabel(), fontsize = 10, rotation = 45)
pair_diagram

In [ ]:
dataset_90.hist(figsize = (20, 20), bins = 20, color = 'green')

In [ ]:
dataset_0.hist(figsize = (20, 20), bins = 20, color = 'green')